# Final Project Phase 2 Summary
This Jupyter Notebook (.ipynb) will serve as the skeleton file for your submission for Phase 2 of the Final Project. Answer all statements addressed below as specified in the instructions for the project, covering all necessary details. Please be clear and concise in your answers. Each response should be at most 3 sentences. Good luck! <br><br>

Note: To edit a Markdown cell, double-click on its text.

## Jupyter Notebook Quick Tips
Here are some quick formatting tips to get you started with Jupyter Notebooks. This is by no means exhaustive, and there are plenty of articles to highlight other things that can be done. We recommend using HTML syntax for Markdown but there is also Markdown syntax that is more streamlined and might be preferable.
<a href = "https://towardsdatascience.com/markdown-cells-jupyter-notebook-d3bea8416671">Here's an article</a> that goes into more detail. (Double-click on cell to see syntax)

# Heading 1
## Heading 2
### Heading 3
#### Heading 4
<br>
<b>BoldText</b> or <i>ItalicText</i>
<br> <br>
Math Formulas: $x^2 + y^2 = 1$
<br> <br>
Line Breaks are done using br enclosed in < >.
<br><br>
Hyperlinks are done with: <a> https://www.google.com </a> or
<a href="http://www.google.com">Google</a><br>

# Data Collection and Cleaning
You are required to provide data collection and cleaning for the three (3) minimum datasets. Create a function for each of the following sections that reads or scrapes data from a file or website, manipulate and cleans the parsed data, and writes the cleaned data into a new file.

Make sure your data cleaning and manipulation process is not too simple. Performing complex manipulation and using modules not taught in class shows effort, which will increase the chance of receiving full credit.


## Data Sources
Include sources (as links) to your datasets. Add any additional data sources if needed. Clearly indicate if a data source is different from one submitted in your Phase I, as we will check that it satisfies the requirements.
*   Downloaded Dataset Source: https://catalog.data.gov/dataset/heart-disease-mortality-data-among-us-adults-35-by-state-territory-and-county-2019-2021

*   Web Collection #1 Source: https://nccd.cdc.gov/DHDSPAtlas/Reports.aspx
*   Web Collection #2 Source: https://apps.bea.gov/api/data/?UserID=80BEF102-7B1D-4714-BA8D-7D4D45AC07A9&method=GetData&datasetname=Regional&TableName=CAGDP2&LineCode=1&Year=2020&GeoFips=COUNTY&ResultFormat=json  

*   Web Collection #3 Source: https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_income

## Downloaded Dataset Requirement

Fill in the predefined functions with your data scraping/parsing code. You may modify/rename each function as you seem fit, but you must provide at least 3 separate functions that clean each of your required datasets.


In [ ]:
def data_parser():
  import pandas as pd
  import numpy as np

  ddf = pd.read_csv("Heart_Disease_Mortality_Data_Among_US_Adults__35___by_State_Territory_and_County___2019-2021.csv")

  ddf.rename(columns = {"Stratification1" : "Gender"},inplace = True)
  ddf.rename(columns = {"Stratification2" : "Ethnicity"},inplace = True)
  ddf.rename(columns = {"Data_Value" : "Deaths Per 100,000 population"},inplace = True)
  ddf = ddf.drop(["StratificationCategory1","StratificationCategory2","TopicID","LocationID","Y_lat","X_lon","Georeference",
  "Class","Topic","DataSource","Data_Value_Unit"], axis = 1 )
  ddf = ddf.sort_values(by = ["LocationAbbr","GeographicLevel","Ethnicity","Gender"],ascending = [True, False,True,False])
  ddf = ddf.reset_index()
  ddf = ddf.drop("index",axis = 1)

  ethn_avg = ddf.groupby(["Ethnicity"])["Deaths Per 100,000 population"].mean()

  def replacemissing(row):
    if row["Data_Value_Footnote"] == "Insufficient Data":
      try:
        state_avg = ddf[
            (ddf["GeographicLevel"] == "State") &
            (ddf["Ethnicity"] == row["Ethnicity"]) &
            (ddf["Gender"] == row["Gender"]) &
            (ddf["LocationAbbr"] == row["LocationAbbr"])
        ].iloc[0]
        return state_avg["Deaths Per 100,000 population"]
      except IndexError:
        return ethn_avg[row["Ethnicity"]]
    else:
      return row["Deaths Per 100,000 population"]

  ddf["Deaths Per 100,000 population"] = ddf.apply(lambda row: replacemissing(row),axis = 1)
  ddf = ddf[ddf["GeographicLevel"] != "State"]
  ddf = ddf.drop(["Data_Value_Type","Data_Value_Footnote_Symbol","Data_Value_Footnote"], axis = 1)

  ddf.to_csv("cleaned_heart_disease_mortality.csv",index = False)

  return ddf






############ Function Call ############
data_parser()

,Year,LocationAbbr,LocationDesc,GeographicLevel,"Deaths Per 100,000 population",Gender,Ethnicity
24,2020,AK,Nome,County,561.7,Overall,American Indian or Alaska Native
25,2020,AK,Bristol Bay,County,442.5,Overall,American Indian or Alaska Native
26,2020,AK,Southeast Fairbanks,County,326.6,Overall,American Indian or Alaska Native
27,2020,AK,Hoonah-Angoon,County,408.5,Overall,American Indian or Alaska Native
28,2020,AK,Yukon-Koyukuk,County,289.1,Overall,American Indian or Alaska Native
...,...,...,...,...,...,...,...
78787,2020,WY,Goshen County,County,213.0,Female,White
78788,2020,WY,Uinta County,County,311.2,Female,White
78789,2020,WY,Teton County,County,152.8,Female,White
78790,2020,WY,Park County,County,242.5,Female,White


## Web Collection Requirement \#1 (HTML)


In [ ]:
%pip install selenium
%pip install webdriver_manager
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 21.5 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,457 kB]
Get:1

In [13]:
def web_parser1():
  import pandas as pd
  import numpy as np
  import re
  from selenium import webdriver
  from selenium.webdriver.common.by import By
  from selenium.webdriver.support.ui import Select
  from selenium.webdriver.chrome.service import Service
  from selenium.webdriver.chrome.options import Options
  from webdriver_manager.chrome import ChromeDriverManager
  import time

  chrome_options = Options()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument('--disable-gpu')

  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
  url = 'https://nccd.cdc.gov/DHDSPAtlas/Reports.aspx'
  driver.get(url)

  time.sleep(2)

  driver.find_element(By.XPATH, "//a[@href='#tabReport2']").click()

  time.sleep(2)

  select = Select(driver.find_element(By.ID, '2_theme_select'))
  select.select_by_visible_text('Median Household Income ($)')

  time.sleep(2)

  driver.find_element(By.ID, 'btn_2_report_submit').click()

  time.sleep(6) #can possibly lower

  table_element = driver.find_element(By.ID, 'report_sd_table')

  table_html = table_element.get_attribute('outerHTML')

  df = pd.read_html(table_html)[0]

  insuf_table_element = driver.find_element(By.ID, 'insufficientData_sd_table')

  insuf_table_html = insuf_table_element.get_attribute('outerHTML')

  insuf_df = pd.read_html(insuf_table_html)[0]

  driver.quit()

  # return df
  df_income = pd.concat([df, insuf_df], ignore_index = True)
  df_income = df_income.drop('Unnamed: 3', axis = 1)

  ## inconsistency 1 - additional char
  df_income['Category Range'] = df_income['Category Range'].str.replace(r'\(\d\d*\)', '', regex = True)
  # return df_income

  ## inconsistency 3.1 - replace 'Insufficient Data' in Value with NaN
  df_states = extra_source1()
  state_income_map = df_states.set_index('State or territory')['Median household income (2019), ACS[9]'].to_dict()

  def replace_value(row):
    if row['Value'] == 'Insufficient Data':
      income = state_income_map.get(row['State'], np.nan)
      if re.match(r"\$", income):
          income = int(income.replace("$", "").replace(",", ""))
      return income
    return row['Value']

  df_income['Value'] = df_income.apply(replace_value, axis = 1)
  # return df_income

  ## inconsistency 3.2 - replace 'Insufficient Data' in Catagory Range with state avg
  df_states = extra_source1()

  state_income_map = df_states.set_index('State or territory')['Median household income (2019), ACS[9]'].to_dict()

  def replace_category_range(row):
    if row['Category Range'].strip() == 'Insufficient Data':
      return f"{state_income_map.get(row['State'], 'N/A')}"
    return row['Category Range']

  df_income['Category Range'] = df_income.apply(replace_category_range, axis = 1)
  df_income = df_income.drop("Category Range",axis = 1)
  df_income.to_csv("cleaned_median_income.csv",index = False)
  return df_income

############ Function Call ############
web_parser1()

<ipython-input-13-6985c5231789>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]
<ipython-input-13-6985c5231789>:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  insuf_df = pd.read_html(insuf_table_html)[0]
<ipython-input-10-c52b4a8f7576>:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html((str(table)))[2]
<ipython-input-10-c52b4a8f7576>:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html((str(table)))[2]


,County,State,Value
0,McDowell,WV,29000
1,Buffalo,SD,31000
2,East Carroll,LA,31000
3,Holmes,MS,32000
4,Humphreys,MS,33000
...,...,...,...
3221,Yabucoa,PR,20474
3222,Yauco,PR,20474
3223,Saint Croix (County Equivalent),VI,37254
3224,Saint John (County Equivalent),VI,37254


## Web Collection Requirement \#2 (API)


In [ ]:
def web_parser2():
  import requests
  import pandas as pd
  import numpy as np

  url = "https://apps.bea.gov/api/data/?UserID=80BEF102-7B1D-4714-BA8D-7D4D45AC07A9&method=GetData&datasetname=Regional&TableName=CAGDP2&LineCode=1&Year=2020&GeoFips=COUNTY&ResultFormat=json"
  resp = requests.get(url)
  data = resp.json()

  wdf = pd.DataFrame(data['BEAAPI']['Results']['Data'])
  wdf = wdf.drop(["Code","UNIT_MULT","NoteRef","GeoFips"],axis = 1)

  split = wdf["GeoName"].str.split(", ", expand=True)
  wdf["County"] = split[0]
  wdf["State"] = split[1]
  wdf = wdf.drop("GeoName",axis = 1)
  wdf = wdf.iloc[:,[3,4,0,1,2]]
  wdf.to_csv("cleaned_county_gdp.csv",index = False)

  return wdf




############ Function Call ############
web_parser2()

,County,State,TimePeriod,CL_UNIT,DataValue
0,Autauga,AL,2020,Thousands of dollars,1813553
1,Baldwin,AL,2020,Thousands of dollars,8762106
2,Barbour,AL,2020,Thousands of dollars,786529
3,Bibb,AL,2020,Thousands of dollars,501320
4,Blount,AL,2020,Thousands of dollars,973414
...,...,...,...,...,...
3113,Sweetwater,WY,2020,Thousands of dollars,3399440
3114,Teton,WY,2020,Thousands of dollars,2819873
3115,Uinta,WY,2020,Thousands of dollars,823225
3116,Washakie,WY,2020,Thousands of dollars,365071


## Additional Dataset Parsing/Cleaning Functions

Write any supplemental (optional) functions here.

In [ ]:
%pip install us

In [10]:
def extra_source1():
  import pandas as pd
  import numpy as np
  import requests
  from bs4 import BeautifulSoup
  import us

  wiki_url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_income'

  response = requests.get(wiki_url)
  soup = BeautifulSoup(response.content, 'html.parser')

  table = soup.findAll('table', {'class': 'wikitable'})

  df = pd.read_html((str(table)))[2]

  abbv = us.states.mapping('name', 'abbr')
  abbv['Washington, D.C.'] = 'DC'
  df['State or territory'] = df['State or territory'].replace('U.S. Virgin Islands', 'Virgin Islands')
  df['State or territory'] = df['State or territory'].replace(abbv)

  df = df[['State or territory', 'Median household income (2019), ACS[9]']]
  df.to_csv("cleaned_states_median_income.csv", index = False)
  return df

############ Function Call ############
extra_source1()

<ipython-input-10-c52b4a8f7576>:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html((str(table)))[2]


,State or territory,"Median household income (2019), ACS[9]"
0,United States,"$65,712"
1,DC,"$92,266"
2,MA,"$85,843"
3,CT,"$78,833"
4,NJ,"$85,751"
5,MD,"$86,738"
6,NY,"$72,108"
7,WA,"$78,687"
8,NH,"$77,933"
9,CO,"$77,127"


In [ ]:
# Define further extra source functions as necessary

# Inconsistencies

For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. The first inconsistency was removing additional characters (###) at the end of the string for the category range column in the HTML data. These characters are useless and impose on the useful data, so we got rid of them by using .replace() and regex to replace them with nothing.


2. In the downloaded dataset, there are multiple nan values in the Data_Value column which was due to insufficient data to report accurately for that county. In order to deal with this inconsistency we decided that the best course of action was to use the state average for that ethnicity and gender to get the best approximation. However, there were also some states that had insufficient data to report for a specific ethnicity and gender. Therefore for the remaining counties that still had nan values, we decided the best approach was to use the average of that ethnicity specifically to fill in the data.

3. In the HTML dataset, there is  “Insufficient Data” in the Value and Category Range columns. The Value is the data that we will use to analyze with the other datasets, so we decided to replace the "Insufficient Data" with the average state median income via data scraped from a Wikipedia table using a function with if-return statements and .apply(). Initally, we did the same process for the Category Range column, but then decided that the data in the column is not necessary for our project and removed it using .drop().

4. In the additional Wikipedia dataset, the State or territory column had the full name of each state/territory, which made it difficult to add the necessary data to replace the "Insufficient Data". By importing us, we were able to easily change each state name into their abbreviated versions by using masking and .replace().


5. While working with the API we noticed that one of the values that it was reporting out on was the GeoName of that area. However, this GeoName contained both the county and the state which was inconsistent with all of our other datasets which had the county and state separate. Therefore, we decided that it would be best to split this column up based on the comma, and return two separate columns named County and State to better match up with our other datasets for later analysis.
